In [10]:
import os
from OA_utils.OAPreprocessingScripts import *
from OA_utils.OpenSimScripts import *

# Preprocess OA data

In [8]:
root_dir = '/Users/briankeller/Desktop/GRFMuscleModel/Old_Young_Walking_Data/'
OA_subjects = [f'OA{i}' for i in range(1, 26)]
Y_subjects = [f'Y{i}' for i in range(1, 23)]
subjects = OA_subjects + Y_subjects
bad_subjects = ['OA3', 'OA6', 'OA15', 'OA16', 'OA21', 'Y3']
trial_names = []
for bs in bad_subjects:
    subjects.remove(bs)
speeds = ['80', '100', '120']
n_trials = 5
subject_trials = {}
for subj in subjects:
    subj_dir = os.path.join(root_dir, subj, 'Walking/Files_W_HJCs/')
    output_dir = os.path.join(root_dir, 'transformed/')
    # Initialize nested dict
    if subj[0] == 'O':
        subject_trials[subj] = {
            'static': {
                'input': os.path.join(subj_dir, f'{subj}_walk_static1.trc'),
                'output': os.path.join(output_dir, f'{subj}_walk_static1_transformed.trc')
            },
            'tracking': [],
            'forces': [],
            
        }
    else:
        subject_trials[subj] = {
            'static': {
                'input': os.path.join(subj_dir, f'{subj}_walking_static1.trc'),
                'output': os.path.join(output_dir, f'{subj}_walk_static1_transformed.trc')
            },
            'tracking': [],
            'forces': [],
            
        }
    # Populate tracking and force trial lists
    for spd in speeds:
        for i in range(1, n_trials + 1):
            trial_name = f'{subj}_{spd}_{i}'
            trial_names.append(str(trial_name))
            tracking_in = os.path.join(subj_dir, f'{trial_name}.trc')
            tracking_out = os.path.join(output_dir, f'{trial_name}_transformed.trc')
            force_in = os.path.join(subj_dir, f'{trial_name}.forces')
            force_out = os.path.join(output_dir, f'{trial_name}_transformed.mot')

            subject_trials[subj]['tracking'].append({'input': tracking_in, 'output': tracking_out})
            subject_trials[subj]['forces'].append({'input': force_in, 'output': force_out})
all_segs = {}
for subj, data in subject_trials.items():
    process_hjc_trc(input_path=data['static']['input'], 
                    output_path=data['static']['output'], 
                    markers_to_drop=[])
    for trc, forces in zip(data['tracking'], data['forces']):
        trial_segs = preprocess_trc_grf(trc_ip=trc['input'], 
                           trc_op=trc['output'],
                           markers_to_drop=[], 
                           grf_ip=forces['input'], 
                           grf_op=forces['output'],
                           grf_pickle_path='')
        for trial_name, seg_dict in trial_segs.items():
            subj_name = trial_name.split('_')[0]  
            if subj_name not in all_segs:
                all_segs[subj_name] = {}
            all_segs[subj_name][trial_name] = seg_dict
print(all_segs)

{'OA1': {'OA1_80_1': {'left': [(1.2885, 2.0735)], 'right': [(0.6945, 1.475), (1.886, 2.696)]}, 'OA1_80_2': {'left': [(1.207, 2.011)], 'right': [(0.6065, 1.4145), (1.824, 2.6345)]}, 'OA1_80_3': {'left': [(1.151, 1.9685)], 'right': [(0.543, 1.357), (1.778, 2.577)]}, 'OA1_80_4': {'left': [(1.232, 2.0545)], 'right': [(0.624, 1.433), (1.8555, 2.689)]}, 'OA1_80_5': {'left': [(1.0405, 1.837)], 'right': [(0.4395, 1.2235), (1.652, 2.4615)]}, 'OA1_100_1': {'left': [(0.912, 1.6125)], 'right': [(0.37, 1.072), (1.4565, 2.1605)]}, 'OA1_100_2': {'left': [(1.0515, 1.741)], 'right': [(0.5325, 1.214), (1.5905, 2.278)]}, 'OA1_100_3': {'left': [(0.978, 1.6435)], 'right': [(0.476, 1.1395), (1.502, 2.178)]}, 'OA1_100_4': {'left': [(0.551, 1.1595), (1.5375, 2.1985)], 'right': [(1.0135, 1.698)]}, 'OA1_100_5': {'left': [(0.8995, 1.5655)], 'right': [(0.385, 1.061), (1.418, 2.1145)]}, 'OA1_120_1': {'left': [(0.9025, 1.513)], 'right': [(0.409, 1.0285)]}, 'OA1_120_2': {'left': [(0.7965, 1.4055)], 'right': [(0.347,

# Scale Generic Models

In [12]:
print(subject_trials.keys())

dict_keys(['OA1', 'OA2', 'OA4', 'OA5', 'OA7', 'OA8', 'OA9', 'OA10', 'OA11', 'OA12', 'OA13', 'OA14', 'OA17', 'OA18', 'OA19', 'OA20', 'OA22', 'OA23', 'OA24', 'OA25', 'Y1', 'Y2', 'Y4', 'Y5', 'Y6', 'Y7', 'Y8', 'Y9', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17', 'Y18', 'Y19', 'Y20', 'Y21', 'Y22'])


In [14]:
OA_subj_masses = [63.5026, 61.68824, 68.0385, 68.0385, 59.87388, 76.65671, 57.15234, 71.66722,
               84.36774, 67.13132, 52.61644, 70.76004, 75.29594, 69.39927, 83.00697, 47.62695,
              61.68824, 84.82133, 74.38876, 83.91415]
Y_subj_masses = [63, 85, 54, 60, 54, 83, 53, 62, 72, 65, 84, 83, 64, 59, 55, 73, 90, 75, 65, 68]
subj_masses = OA_subj_masses + Y_subj_masses
root_dir = '/Users/briankeller/Desktop/GRFMuscleModel/Old_Young_Walking_Data/'
for subj, data in subject_trials.items():
    scale_generic(root_dir= root_dir, mass=subj_masses[i], static_pose_filename=data['static']['output'])

[info] Processing subject OA1_scaled...
[info] Step 1: Loading generic model
[info] Updating Model file from 40000 to latest format...
[info] Loaded model RajagopalModifiedGeneric from file /Users/briankeller/Desktop/GRFMuscleModel/Old_Young_Walking_Data/Models/RajagopalModified_generic.osim
[warning] Couldn't find file 'r_pelvis.vtp'.
[warning] Couldn't find file 'l_pelvis.vtp'.
[warning] Couldn't find file 'sacrum.vtp'.
[warning] Couldn't find file 'r_femur.vtp'.
[warning] Couldn't find file 'r_tibia.vtp'.
[warning] Couldn't find file 'r_fibula.vtp'.
[warning] Couldn't find file 'r_patella.vtp'.
[warning] Couldn't find file 'r_talus.vtp'.
[warning] Couldn't find file 'r_foot.vtp'.
[warning] Couldn't find file 'r_bofoot.vtp'.
[warning] Couldn't find file 'l_femur.vtp'.
[warning] Couldn't find file 'l_tibia.vtp'.
[warning] Couldn't find file 'l_fibula.vtp'.
[warning] Couldn't find file 'l_patella.vtp'.
[warning] Couldn't find file 'l_talus.vtp'.
[warning] Couldn't find file 'l_foot.vtp

Analyze Scaling Output (O/p of previous cell must be saved to .txt)

In [16]:
parsed = parse_combined_scaling_output('/Users/briankeller/Desktop/GRFMuscleModel/Old_Young_Walking_Data/Combined_scaling_log.txt')
for subject, info in parsed.items():
    rms = info['marker_error_rms']
    max = info['marker_error_max']
    marker = info['marker_error_max_marker']
    print(f'{subject} had marker error rms {rms} and max marker error {max} at the {marker} marker')

Read scaling log
OA1 had marker error rms 0.0173909 and max marker error 0.0286739 at the R.ASIS marker
OA2 had marker error rms 0.0158385 and max marker error 0.0313693 at the L.Knee marker
OA4 had marker error rms 0.0118704 and max marker error 0.0194851 at the R.Heel marker
OA5 had marker error rms 0.0157582 and max marker error 0.0248381 at the L.Heel marker
OA7 had marker error rms 0.0136973 and max marker error 0.0251258 at the L.Knee marker
OA8 had marker error rms 0.0170719 and max marker error 0.0314723 at the R.Knee marker
OA9 had marker error rms 0.0128961 and max marker error 0.0308113 at the L.Knee marker
OA10 had marker error rms 0.0155663 and max marker error 0.0267569 at the L.Knee marker
OA11 had marker error rms 0.0161557 and max marker error 0.031944 at the L.Heel marker
OA12 had marker error rms 0.0145054 and max marker error 0.0328413 at the L.Knee marker
OA13 had marker error rms 0.0171385 and max marker error 0.0319082 at the L.Heel marker
OA14 had marker error r

# Inverse Kinematics?

Needs to be done in .py file to not crash the notebook

Parse IK text logs

In [19]:
ik_df = parse_full_ik_log('/Users/briankeller/Desktop/GRFMuscleModel/Old_Young_Walking_Data/Combined_ik_log.txt', trial_names)
abs_max_count, mean_max_count, mean_rms_count = 0, 0 ,0
problem_trials = []
for idx, row in ik_df.iterrows():
    trial = row['trial_name']
    mean_rms = row['mean_rms']
    std_rms = row['std_rms']
    max_rms = row['max_rms']
    mean_max = row['mean_max']
    std_max = row['std_max']
    max_error = row['max_marker_error']
    max_error_marker = row['max_marker_name']
    #print(f'{trial} had mean rms {mean_rms}, std rms {std_rms}, and max rms{max_rms}. The average max error was{mean_max}, with a standard deviation {std_max} and overall max {max_error} at the marker {max_error_marker}')
    if mean_rms > 0.03:
        #print(f'{trial} had mean RMS greater than 3 cm, it was {mean_rms}')
        mean_rms_count += 1
        problem_trials.append(trial)
    elif mean_max > 0.04:
        #print(f'{trial} had mean max error greater than 4 cm, it was {mean_max}')
        mean_max_count += 1
        problem_trials.append(trial)
    # elif max_error > 0.05:
    #     abs_max_count += 1
    #     print(f'{trial} had max_error greater than 6 cm, it was {max_error} at the {max_error_marker} marker')
    #     problem_trials.append(trial)
print(f'{mean_rms_count} trials with Mean RMS > 3 cm')
print(f'{mean_max_count} trials with Mean max error > 4 cm')
#print(f'{abs_max_count} trials with overall max error > 6 cm')

Read inverse kinematics log
30 trials with Mean RMS > 3 cm
19 trials with Mean max error > 4 cm


In [20]:
print(problem_trials)

['OA5_80_1', 'OA5_80_2', 'OA5_80_3', 'OA5_80_4', 'OA5_100_4', 'OA5_100_5', 'OA5_120_1', 'OA5_120_4', 'OA5_120_5', 'OA11_80_1', 'OA11_80_2', 'OA11_80_3', 'OA11_80_4', 'OA11_80_5', 'OA11_100_2', 'OA11_100_3', 'OA11_100_4', 'OA11_100_5', 'OA11_120_1', 'OA11_120_2', 'OA11_120_3', 'OA11_120_4', 'OA11_120_5', 'OA12_80_2', 'OA12_100_3', 'OA17_80_1', 'OA17_100_1', 'OA17_100_2', 'OA17_100_3', 'OA17_100_4', 'OA17_100_5', 'OA19_80_2', 'Y2_80_1', 'Y2_80_4', 'Y2_80_5', 'Y2_100_3', 'Y2_120_1', 'Y2_120_2', 'Y2_120_3', 'Y2_120_4', 'Y2_120_5', 'Y5_100_3', 'Y10_80_5', 'Y10_100_1', 'Y10_100_3', 'Y12_80_3', 'Y12_100_3', 'Y15_120_3', 'Y17_80_4']


# Inverse Dynamics

In [21]:
root_dir = '/Users/briankeller/Desktop/GRFMuscleModel/Old_Young_Walking_Data/'
os.chdir(root_dir)
for subj, data in subject_trials.items():
    model = osim.Model(f'Results/Scaling/{subj}_scaled.osim')
    for trc, forces in zip(data['tracking'], data['forces']):
        inverse_dynamics(root_dir=root_dir, force_data_filepath=forces['output'], tracking_data_filepath=trc['output'], model=model)

[info] Loaded model OA1_scaled from file Results/Scaling/OA1_scaled.osim
[warning] Couldn't find file 'r_pelvis.vtp'.
[warning] Couldn't find file 'l_pelvis.vtp'.
[warning] Couldn't find file 'sacrum.vtp'.
[warning] Couldn't find file 'r_femur.vtp'.
[warning] Couldn't find file 'r_tibia.vtp'.
[warning] Couldn't find file 'r_fibula.vtp'.
[warning] Couldn't find file 'r_patella.vtp'.
[warning] Couldn't find file 'r_talus.vtp'.
[warning] Couldn't find file 'r_foot.vtp'.
[warning] Couldn't find file 'r_bofoot.vtp'.
[warning] Couldn't find file 'l_femur.vtp'.
[warning] Couldn't find file 'l_tibia.vtp'.
[warning] Couldn't find file 'l_fibula.vtp'.
[warning] Couldn't find file 'l_patella.vtp'.
[warning] Couldn't find file 'l_talus.vtp'.
[warning] Couldn't find file 'l_foot.vtp'.
[warning] Couldn't find file 'l_bofoot.vtp'.
[warning] Couldn't find file 'hat_spine.vtp'.
[warning] Couldn't find file 'hat_jaw.vtp'.
[warning] Couldn't find file 'hat_skull.vtp'.
[warning] Couldn't find file 'hat_ri